"""
An example to demonstrate online linear system identification

We demonstrate the use of OnlineLinearModel class with a simple linear system.
Take a 2D time-varying system dx/dt=A(t)x(t)+B(t)u(t), where A(t) and B(t)
are slowly varying with time. In particular, we take A(t)=(1+eps*t)*A,
B(t)=(1+eps*t)*B, and eps = 0.1 is small. It is discretize with
time step dt = 0.1. Denote the discrete system as x(k)=A(k)x(k-1)+
B(k)u(k-1).

At time step k+1, we need to include new snapshot pair x(k-1), u(k-1), x(k).
We would like to update the adaptive model in real-time

Authors: 
Hao Zhang

References:
Zhang, Hao, Clarence W. Rowley, Eric A. Deem, and Louis N. Cattafesta.
"Online dynamic mode decomposition for time-varying systems."
SIAM Journal on Applied Dynamical Systems 18, no. 3 (2019): 1586-1609.

Created:
June 2017.
"""

In [80]:
!pip install -r requirements.txt

In [81]:
!pip list

Package                            Version             Location
---------------------------------- ------------------- ---------------------------------------
alabaster                          0.7.12
anaconda-client                    1.7.2
anaconda-navigator                 1.10.0
anaconda-project                   0.8.3
appdirs                            1.4.4
applaunchservices                  0.2.1
appnope                            0.1.0
appscript                          1.1.1
argh                               0.26.2
argon2-cffi                        20.1.0
asn1crypto                         1.4.0
astroid                            2.4.2
astropy                            4.0.2
async-generator                    1.10
atomicwrites                       1.4.0
attrs                              20.3.0
autopep8                           1.5.4
Babel                              2.8.1
backcall                           0.2.0
backports.functools-lru-cache      1.6.1
backports.shutil-

In [98]:
!pip install slycot

     |████████████████████████████████| 1.5 MB 2.0 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  ERROR: Command errored out with exit status 1:
   command: /Users/haozhang/opt/anaconda3/bin/python /Users/haozhang/opt/anaconda3/lib/python3.8/site-packages/pip/_vendor/pep517/_in_process.py build_wheel /var/folders/wk/vrj9fw_x0s7dqj29gfkhgf2m0000gn/T/tmpvgen0b4y
       cwd: /private/var/folders/wk/vrj9fw_x0s7dqj29gfkhgf2m0000gn/T/pip-install-h9fa_oxv/slycot
  Complete output (86 lines):
  Not searching for unused variables given on the command line.
  CMake Error: CMake was unable to find a build program corresponding to "Ninja".  CMAKE_MAKE_PROGRAM is not set.  You probably need to select a different build tool.
  -- Configuring incomplete, errors occurred!
  See also "/private/var/folders/wk/vrj9fw_x0s7dqj29gfkhgf2m0000gn/T/pip-install-h9fa_oxv/slycot/_cmake_test_compile/build/CMakeFiles/C

In [99]:
"""
An example to demonstrate online linear system identification

We demonstrate the use of OnlineLinearModel class with a simple linear system.
Take a 2D time-varying system dx/dt=A(t)x(t)+B(t)u(t), where A(t) and B(t)
are slowly varying with time. In particular, we take A(t)=(1+eps*t)*A,
B(t)=(1+eps*t)*B, and eps = 0.1 is small. It is discretize with
time step dt = 0.1. Denote the discrete system as x(k)=A(k)x(k-1)+
B(k)u(k-1).

At time step k+1, we need to include new snapshot pair x(k-1), u(k-1), x(k).
We would like to update the adaptive model in real-time

Authors: 
Hao Zhang

References:
Zhang, Hao, Clarence W. Rowley, Eric A. Deem, and Louis N. Cattafesta.
"Online dynamic mode decomposition for time-varying systems."
SIAM Journal on Applied Dynamical Systems 18, no. 3 (2019): 1586-1609.

Created:
June 2017.
"""


import matplotlib.pyplot as plt
import numpy as np
from osysid import OnlineLinearModel
from control import lqr, StateSpace, ctrb

# define dynamics
# https://en.wikipedia.org/wiki/Harmonic_oscillator#Damped_harmonic_oscillator
A = np.array([[0, 1], [-1, 0.1]])
B = np.array([[0], [1]])
epsilon = 1e-2

# controllability
Ctrb = ctrb(A, B)
assert np.linalg.matrix_rank(Ctrb) == A.shape[0]

# time-varying dynamics
def dyn(t, x, u):
    At = (1 + epsilon * t) * A
    Bt = (1 + epsilon * t) * B
    dxdt = At.dot(x) + Bt.dot(u)
    return dxdt


# set up simulation parameter
dt = 0.1
tmax, tc = 20, 0.5
T, kc = int(tmax / dt), int(tc / dt)
tspan = np.linspace(0, tmax, T + 1)

# dimensions
n = 2
k = 1

# online linear system identification setup
alpha = 0.01 ** (2.0 / kc)
olm = OnlineLinearModel(n, k, None, alpha)

# store data mtrices
x = np.zeros([n, T])
u = np.zeros([k, T])

# initial condition, state and control
x[:, 0] = np.array([1, 0])
u[:, 0] = np.array([0])

# system simulation
for t in range(1, T):
    # forward the system for one step
    x[:, t] = x[:, t - 1] + dt * dyn(t * dt, x[:, t - 1], u[:, t - 1])
    # use new measurement to update online system identification
    olm.update(x[:, t - 1], u[:, t - 1], x[:, t])
    if t > 2 * max(n, n + k):
        # get LQR gain, output = state
        sys = StateSpace(olm.A, olm.B, np.eye(n), np.zeros((n, k)), dt=True)
        K, S, E = lqr(sys, 4 * np.eye(n), np.eye(k))
        # apply control
        u[:, t] = -1.0 * K.dot(x[:, t])
        print("lqr control input")
    else:
        u[:, t] = np.random.randn()
        print("random input")


# visualize snapshots
# plt.figure(figsize=(10, 6))
# plt.rc("text", usetex=True)
# plt.rc("font", family="serif")
# plt.plot(tspan[1:], x[0, :], "b-", linewidth=1.0, label="$x_1(t)$")
# plt.plot(tspan[1:], x[1, :], "g-", linewidth=1.0, label="$x_2(t)$")
# plt.plot(tspan[1:], u.reshape(-1), "r-", linewidth=1.0, label="$u(t)$")
# plt.legend(loc="best", fontsize=12, shadow=True)
# plt.xlabel("Time", fontsize=12)
# plt.title("State and control", fontsize=12)
# plt.tick_params(labelsize=12)
# plt.grid()
# plt.show()



fig, axs = plt.subplots(2, figsize=(10, 6))
fig.suptitle('State and control')
axs[0].plot(tspan[1:], x[0, :], "b-", linewidth=1.0, label="$x_1(t)$")
axs[0].plot(tspan[1:], x[1, :], "g-", linewidth=1.0, label="$x_2(t)$")

axs[1].plot(tspan[1:], u.reshape(-1), "r-", linewidth=1.0, label="$u(t)$")

random input
random input
random input
random input
random input
random input


ControlSlycot: can't find slycot module 'sb02md' or 'sb02nt'

In [114]:
olm.C

array([[ 1.0000485 ,  0.10065413],
       [-0.10004525,  1.01056801]])

In [115]:
olm.D

array([[-1.85806653e-05],
       [ 1.00455137e-01]])

In [100]:
from slycot import sb02mt

ModuleNotFoundError: No module named 'slycot'

In [101]:
import slycot

ModuleNotFoundError: No module named 'slycot'

In [102]:
!pip install slycot

  Using cached slycot-0.4.0.0.tar.gz (1.5 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  ERROR: Command errored out with exit status 1:
   command: /Users/haozhang/opt/anaconda3/bin/python /Users/haozhang/opt/anaconda3/lib/python3.8/site-packages/pip/_vendor/pep517/_in_process.py build_wheel /var/folders/wk/vrj9fw_x0s7dqj29gfkhgf2m0000gn/T/tmpkgd1zti_
       cwd: /private/var/folders/wk/vrj9fw_x0s7dqj29gfkhgf2m0000gn/T/pip-install-dzn8rlh1/slycot
  Complete output (86 lines):
  Not searching for unused variables given on the command line.
  CMake Error: CMake was unable to find a build program corresponding to "Ninja".  CMAKE_MAKE_PROGRAM is not set.  You probably need to select a different build tool.
  -- Configuring incomplete, errors occurred!
  See also "/private/var/folders/wk/vrj9fw_x0s7dqj29gfkhgf2m0000gn/T/pip-install-dzn8rlh1/slycot/_cmake_test_compile/build/CMakeFiles/CMakeOutput.log".
  Not

In [77]:
w, v = np.linalg.eig(A)

In [78]:
w

array([-0.1+0.99498744j, -0.1-0.99498744j])

In [85]:
Ctrb = ctrb(A, B)

In [ ]:
Ctrb

In [87]:
A

array([[ 0. ,  1. ],
       [-1. , -0.2]])

In [104]:
A

array([[ 0. ,  1. ],
       [-1. ,  0.1]])

In [105]:
B

array([[0],
       [1]])

In [106]:
C = np.eye(2)
C

array([[1., 0.],
       [0., 1.]])

In [109]:
D = np.zeros((2, 1))
D

array([[0.],
       [0.]])

In [113]:
help(StateSpace)

Help on class StateSpace in module control.statesp:

class StateSpace(control.lti.LTI)
 |  StateSpace(*args, **kw)
 |  
 |  StateSpace(A, B, C, D[, dt])
 |  
 |  A class for representing state-space models
 |  
 |  The StateSpace class is used to represent state-space realizations of linear
 |  time-invariant (LTI) systems:
 |  
 |      dx/dt = A x + B u
 |          y = C x + D u
 |  
 |  where u is the input, y is the output, and x is the state.
 |  
 |  The main data members are the A, B, C, and D matrices.  The class also
 |  keeps track of the number of states (i.e., the size of A).  The data
 |  format used to store state space matrices is set using the value of
 |  `config.defaults['use_numpy_matrix']`.  If True (default), the state space
 |  elements are stored as `numpy.matrix` objects; otherwise they are
 |  `numpy.ndarray` objects.  The :func:`~control.use_numpy_matrix` function
 |  can be used to set the storage type.
 |  
 |  Discrete-time state space system are implemented

In [112]:
sys = StateSpace(A, B, C, D)

In [111]:
K = lqr(sys, np.eye(2), np.eye(1))

ControlSlycot: can't find slycot module 'sb02md' or 'sb02nt'